In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from pathlib import Path

# Frame-wise optical flow
def calculate_optical_flow(prev_gray, gray):
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None,
                                        0.5, 3, 15, 3, 5, 1.2, 0)
    return np.mean(flow), np.std(flow)

# Edge ratio per frame
def calculate_edge_ratio(gray):
    edges = cv2.Canny(gray, 100, 200)
    edge_pixels = np.count_nonzero(edges)
    total_pixels = gray.shape[0] * gray.shape[1]
    return edge_pixels / total_pixels if total_pixels != 0 else 0

# Keypoints per frame
sift = cv2.SIFT_create()
def count_keypoints(gray):
    kp, _ = sift.detectAndCompute(gray, None)
    return len(kp)

# FFT peak per frame
def extract_fft_peak(gray):
    fft = np.fft.fft2(gray)
    fft_shift = np.fft.fftshift(fft)
    fft_magnitude = np.abs(fft_shift)
    return np.max(fft_magnitude)

# Extract per-frame features for a video
def extract_per_frame_features(video_path, state):
    cap = cv2.VideoCapture(video_path)
    video_name = os.path.basename(video_path)
    view = 'angle' if 'angle' in video_name.lower() else 'front'

    frame_index = 0
    ret, prev_frame = cap.read()
    if not ret:
        cap.release()
        return []

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    data = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        mean_flow, std_flow = calculate_optical_flow(prev_gray, gray)
        edge_ratio = calculate_edge_ratio(gray)
        keypoints = count_keypoints(gray)
        fft_peak = extract_fft_peak(gray)

        data.append([
            video_name, frame_index, mean_flow, std_flow, edge_ratio,
            keypoints, fft_peak, view, state
        ])

        prev_gray = gray
        frame_index += 1

    cap.release()
    return data

# Directory setup
base_dir = Path('evm_segmented_videos')
output_dir = Path('Datasets')
states = ['Bearing_fault', 'Normal_state', 'Unbalance_weight']
segments = ['segmented_5', 'segmented_5_overlap', 'segmented_10',
            'segmented_10_overlap', 'segmented_15', 'segmented_15_overlap']

for state in states:
    (output_dir / state).mkdir(parents=True, exist_ok=True)

# Process each state
for state in states:
    all_data = []
    for segment in segments:
        segment_path = base_dir / state / segment
        video_paths = glob(str(segment_path / "*.avi"))
        if not video_paths:
            continue

        for video_path in tqdm(video_paths, desc=f"{state} - {segment}"):
            frame_data = extract_per_frame_features(video_path, state)
            all_data.extend(frame_data)

    if all_data:
        columns = ['video_name', 'frame_index', 'mean_flow', 'std_flow',
                   'edge_ratio', 'keypoint_count', 'fft_peak', 'view', 'state']
        df = pd.DataFrame(all_data, columns=columns)
        output_csv = output_dir / state / f"{state}_framewise_features.csv"
        df.to_csv(output_csv, index=False)

print("✅ Frame-wise feature extraction completed!")


Processing Bearing_fault:  39%|███▉      | 28/72 [02:24<03:46,  5.15s/it]


KeyboardInterrupt: 